# 수어(Sign Language) 모델 

### 데이터셋 설명
- 데이터 출처 : https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=103 
- 데이터 구성 : 총 536000 수어영상클립(.MP4파일),총 536,000 수어 영상에 대한 형태소 및 비수지 요소 가공값(json 파일),총 536,000 수어 영상에 대한 30fps 분할 이미지에 대한 keypoint 값(json 파일)
- 기타 설명 : 수어 영상에 대한 30fps 분할 이미지에 대한 keypoint 값을 이용했습니다.

### 1. 필요한 라이브러리 로드

In [2]:
import os
import re
import glob
import json
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split

from keras.optimizers import Adam

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping

### 2-1 정규화 함수
- sklearn에서 제공하는 정규화 함수 사용

In [3]:
#정규화 
def min_max_scaling(data):
    min_val = np.min(data)
    max_val = np.max(data)
    data_normalized = (data - min_val) / (max_val - min_val)
    return data_normalized

### 2-2 각 json 파일에서 face,pose,handr,handl의 값들을 저장합니다.값들이 x,y,정확도를 저장하므로, 필요없는 정확도 값은 없애고 x,y,만 저장하는 함수

In [4]:
def change_numpy(json_data):
    people = json_data.get("people",{})
    face_keypoints_2d = np.array(people.get("face_keypoints_2d",[]))
    pose_keypoints_2d = np.array(people.get("pose_keypoints_2d",[]))
    hand_left_keypoints_2d = np.array(people.get("hand_left_keypoints_2d",[]))
    hand_right_keypoints_2d = np.array(people.get("hand_right_keypoints_2d",[]))

    #X,Y,정확도 값중에 정확도 값을 없애고 X,Y값만 저장함
    face_keypoints_2d = face_keypoints_2d.reshape(-1, 3)[:, :2]
    pose_keypoints_2d = pose_keypoints_2d.reshape(-1, 3)[:, :2]
    hand_left_keypoints_2d = hand_left_keypoints_2d.reshape(-1, 3)[:, :2]
    hand_right_keypoints_2d = hand_right_keypoints_2d.reshape(-1, 3)[:, :2]

    return face_keypoints_2d, pose_keypoints_2d, hand_left_keypoints_2d, hand_right_keypoints_2d

# 3. 데이터셋 dataset_list를 이용함

In [5]:
# 데이터파일 위치
file_list = './dataset_json/WORD/keypoint'
dataset = [file for file in os.listdir(file_list) if not file.startswith('.DS_Store')]

# label_dict = [1501,1502,1503,1504,1505]
label_dict = [1501]
sequence_length = 30
sequences, labels = [], []

label_map = {1501: 0, 1502: 1,1503: 2,1504: 3,1505: 4}

In [9]:
#폴더 돌면서 데이터 받아오기 
for label in label_dict:
    folder_prefix = f"NIA_SL_WORD{label:04d}_REAL01"
    filtered_folders = [folder for folder in os.listdir(file_list) if folder.startswith(folder_prefix)]

    for folder in filtered_folders:
        print(f'Folder : {folder}\n')
        folder_path = os.path.join(file_list, folder)
        sub_paths = sorted(os.listdir(folder_path))
        last_char = folder[-1]

        for sequence in range(len(sub_paths)):
            window = []
            for frame_num in range(sequence_length):
                json_file_path = os.path.join(folder_path, f"NIA_SL_WORD{label:04d}_REAL01_{last_char}_000000000{frame_num:03d}_keypoints.json")
                print(json_file_path)
                with open(json_file_path, "r") as json_file:
                    json_data = json.load(json_file)
                    keypoints_list = change_numpy(json_data)
#                     print(keypoints_list)
#                     print('')
                    flattened_keypoints = np.concatenate(keypoints_list, axis=0)
                    window.append(flattened_keypoints.reshape(-1))
#             print(window)
            sequences.append(window)
            labels.append(label_map[label])


Folder : NIA_SL_WORD1501_REAL01_D

./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000000_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000001_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000002_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000003_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000004_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000005_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000006_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000007_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000008_keypoints.json
./

./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000021_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000022_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000023_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000024_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000025_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000026_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000027_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000028_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000029_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_W

./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000009_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000010_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000011_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000012_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000013_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000014_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000015_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000016_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000017_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_W

./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000003_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000004_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000005_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000006_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000007_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000008_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000009_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000010_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000011_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_W

./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000005_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000006_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000007_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000008_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000009_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000010_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000011_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000012_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_D\NIA_SL_WORD1501_REAL01_D_000000000013_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_W

...

./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000003_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000004_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000005_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000006_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000007_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000008_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000009_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000010_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000011_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_W

./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000020_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000021_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000022_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000023_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000024_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000025_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000026_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000027_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000028_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_W

./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000002_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000003_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000004_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000005_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000006_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000007_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000008_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000009_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000010_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_W

./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000022_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000023_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000024_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000025_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000026_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000027_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000028_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000029_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000000_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_W

./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000022_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000023_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000024_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000025_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000026_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000027_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000028_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000029_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_WORD1501_REAL01_U\NIA_SL_WORD1501_REAL01_U_000000000000_keypoints.json
./dataset_json/WORD/keypoint\NIA_SL_W

### sequences 설명 추가

In [6]:
np.array(sequences).shape

(3155, 30, 274)

In [7]:
np.array(labels).shape

(3155,)

# 4. 모델 학습 : 데이터셋을 train

In [8]:
X = np.array(sequences)
y = to_categorical(labels).astype(int) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

X_train_normalized = min_max_scaling(X_train)
X_test_normalized = min_max_scaling(X_test)

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 274)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(label_map), activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
early_stopping = EarlyStopping(patience=5, monitor='val_categorical_accuracy', mode='max', restore_best_weights=True)
model.fit(X_train_normalized, y_train, epochs=100, validation_split=0.1, callbacks=[early_stopping])


Epoch 1/100
85/85 [==============================] - 11s 73ms/step - loss: 1.6090 - categorical_accuracy: 0.2125 - val_loss: 1.6034 - val_categorical_accuracy: 0.2433
Epoch 2/100
85/85 [==============================] - 5s 64ms/step - loss: 1.6071 - categorical_accuracy: 0.2184 - val_loss: 1.6060 - val_categorical_accuracy: 0.2200
Epoch 3/100
85/85 [==============================] - 6s 67ms/step - loss: 1.6067 - categorical_accuracy: 0.2236 - val_loss: 1.6026 - val_categorical_accuracy: 0.2200
Epoch 4/100
85/85 [==============================] - 6s 65ms/step - loss: 1.6064 - categorical_accuracy: 0.2173 - val_loss: 1.6035 - val_categorical_accuracy: 0.2200
Epoch 5/100
85/85 [==============================] - 6s 66ms/step - loss: 1.6064 - categorical_accuracy: 0.2139 - val_loss: 1.6024 - val_categorical_accuracy: 0.2200
Epoch 6/100
85/85 [==============================] - 6s 68ms/step - loss: 1.6061 - categorical_accuracy: 0.2236 - val_loss: 1.6021 - val_categorical_accuracy: 0.2200


# 5. 모델 저장

In [9]:
model.save('action.h5')

# 6. 모델 정확도 측정 (text 데이터로 수정 완료)

In [10]:
train_predictions = np.argmax(model.predict(X_test_normalized), axis=1)
NUM_CLASSES = len(label_map)

for action in range(NUM_CLASSES):
    action_indices = np.where(y_train[:, action] == 1)[0]
    action_predictions = train_predictions[action_indices]
    action_accuracy = np.sum(action_predictions == action) / len(action_indices)
    print(f"Training Accuracy for {action} action: {action_accuracy}")

5/5 [==============================] - 1s 17ms/step


IndexError: index 160 is out of bounds for axis 0 with size 158

# 7. 파일로 저장된 모델을 로드하여 예측하는 프로그램 작성

In [ ]:
#하는중입니다